![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `meta-llama/llama-3-1-70b-instruct` to create AI service

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for watsonx.ai AI service.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

This notebook aims to demonstrate the application of Chat models, such as `meta-llama/llama-3-1-70b-instruct`, using the tools provided by LangGraph. LangGraph serves as an Agent Orchestrator, enabling the development of graph-based applications that autonomously execute sequences of actions. In these applications, the Large Language Model (LLM) functions as the primary decision-maker, determining the subsequent steps. 

Following this, an AI service is created based on the previously constructed application.


## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Create AI service](#ai_service)
- [Testing AI service's function locally](#testing)
- [Deploy AI service](#deploy)
- [Example of Executing an AI service](#example)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependencies

In [ ]:
!pip install -U "langgraph>0.2,<0.3" | tail -n 1
!pip install -U "ibm_watsonx_ai>=1.1.22" | tail -n 1
!pip install -U "langchain_ibm>=0.3,<0.4" | tail -n 1

### Define the watsonx.ai credentials
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [2]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
)

### Working with spaces

You need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=wx) to create one.

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select watsonx.ai Runtime instance and press **Create**
- Go to **Manage** tab
- Copy `Space GUID` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: assign space ID below

In [3]:
import os

try:
    space_id = os.environ["SPACE_ID"]
except KeyError:
    space_id = input("Please enter your project_id (hit enter): ")

Create an instance of APIClient with authentication details.

In [2]:
from ibm_watsonx_ai import APIClient

api_client = APIClient(credentials=credentials, space_id=space_id)

Specify the `model_id` of the model you will use for the chat with tools.

In [3]:
model_id = "meta-llama/llama-3-1-70b-instruct"

<a id="ai_service"></a>
## Create AI service

The content of this notebook is derived from and based on the material presented in the [Use watsonx, and `mistralai/mistral-large` with support for tools to perform simple calculations](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/deployments/foundation_models/chat/Use%20watsonx%2C%20and%20%60mistral-large%60%20with%20support%20for%20tools%20to%20perform%20simple%20calculations.ipynb) notebook. 

Prepare function which will be deployed using AI service.

In [4]:
def deployable_ai_service(context, space_id=space_id, url=credentials["url"], model_id=model_id, **kwargs):
    
    from ibm_watsonx_ai import APIClient, Credentials
    from langchain_ibm import ChatWatsonx
    from langchain_core.tools import tool
    from langgraph.prebuilt import create_react_agent

    api_client = APIClient(
        credentials=Credentials(url=url, token=context.generate_token()),
        space_id=space_id
    )

    chat = ChatWatsonx(
        watsonx_client=api_client,
        model_id=model_id,
        params={"temperature": 0.1}
    )
    
    @tool
    def add(a: float, b: float) -> float:
        """Add a and b."""
        return a + b

    @tool
    def subtract(a: float, b: float) -> float:
        """Subtract a and b."""
        return a - b

    @tool
    def multiply(a: float, b: float) -> float:
        """Multiply a and b."""
        return a * b

    @tool
    def divide(a: float, b: float) -> float:
        """Divide a and b."""
        return a / b

    tools = [add, subtract, multiply, divide]
    
    graph = create_react_agent(chat, tools=tools)

    def generate(context) -> dict:
        
        api_client.set_token(context.get_token())
   
        payload = context.get_json()
        question = payload["question"]
        
        response = graph.invoke({"messages": [("user", f"{question}")]})
        
        json_messages = [msg.to_json() for msg in response['messages']]
        
        response['messages'] = json_messages
        
        return {"body": response}
    
    def generate_stream(context):
        
        api_client.set_token(context.get_token())
   
        payload = context.get_json()
        question = payload["question"]
        
        for el in graph.stream({"messages": [("user", f"{question}")]}, stream_mode="values"):
            
            json_messages = [msg.to_json() for msg in el['messages']]
            el['messages'] = json_messages
            yield el
        
    return generate, generate_stream

Add a helpful function to print messages from the model.

In [5]:
import json

def print_message(message):
    print(f" ===== {message['id'][-1]} =====")
    if message["id"][-1] == "AIMessage":
        content = message["kwargs"].get("additional_kwargs", message["kwargs"].get("content"))
        if isinstance(content, dict):
            print(json.dumps(content, indent=2) + "\n")
        else:
            print(content + "\n")
    elif message["id"][-1] == "ToolMessage":
        print(message["kwargs"]["name"])
        print(message["kwargs"]["content"] + "\n")
    else:
        print(message["kwargs"]["content"] + "\n")

def ai_services_pretty_print(iter):
    if "body" in iter:
        iter = iter["body"]
    for message in iter["messages"]:
        print_message(message)
        
def ai_services_pretty_print_stream(iter):
    for el in iter:
        try:
            el = json.loads(el)
        except TypeError:
            pass
        print_message(el["messages"][-1])

<a id="testing"></a>
## Testing AI service's function locally

You can test AI service's function locally. Initialise `RuntimeContext` firstly.

In [6]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=api_client)

In [7]:
local_function = deployable_ai_service(context=context)

Prepare request json payload.

In [8]:
context.request_payload_json = {"question": "What is the total sum of the numbers 11, 13, and 20? Remember to always return the final result using the last tool message."}

Execute the `generate` function locally.

In [9]:
resp = local_function[0](context)

In [10]:
ai_services_pretty_print(resp)

 ===== HumanMessage =====
What is the total sum of the numbers 11, 13, and 20? Remember to always return the final result using the last tool message.

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "id": "chatcmpl-tool-a6a9cd6a24154004b8d3aba6c7cc2018",
      "type": "function",
      "function": {
        "name": "add",
        "arguments": "{\"a\": \"11\", \"b\": \"13\"}"
      }
    }
  ]
}

 ===== ToolMessage =====
add
24.0

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "id": "chatcmpl-tool-0adb0b39ec8d4e1897d9b5c7b876afd8",
      "type": "function",
      "function": {
        "name": "add",
        "arguments": "{\"a\": \"24\", \"b\": \"20\"}"
      }
    }
  ]
}

 ===== ToolMessage =====
add
44.0

 ===== AIMessage =====
The total sum of the numbers 11, 13, and 20 is 44.



Execute the `generate_stream` function locally.

In [11]:
response = local_function[1](context)

In [12]:
ai_services_pretty_print_stream(response)

 ===== HumanMessage =====
What is the total sum of the numbers 11, 13, and 20? Remember to always return the final result using the last tool message.

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "id": "chatcmpl-tool-2d29c56e9c05434495713070582eb79d",
      "type": "function",
      "function": {
        "name": "add",
        "arguments": "{\"a\": \"11\", \"b\": \"13\"}"
      }
    }
  ]
}

 ===== ToolMessage =====
add
24.0

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "id": "chatcmpl-tool-a69bd190f74941dca5ff9974579c97ab",
      "type": "function",
      "function": {
        "name": "add",
        "arguments": "{\"a\": \"24\", \"b\": \"20\"}"
      }
    }
  ]
}

 ===== ToolMessage =====
add
44.0

 ===== AIMessage =====
The total sum of the numbers 11, 13, and 20 is 44.



<a id="deploy"></a>
## Deploy AI service

Prepare a configuration file for defining a custom software specification.

In [13]:
config_yml =\
"""
name: python311
channels:
  - empty
dependencies:
  - pip:
    - langgraph==0.2.44
prefix: /opt/anaconda3/envs/python311
"""

with open("config.yaml", "w", encoding="utf-8") as f:
    f.write(config_yml)

In [14]:
base_sw_spec_id = api_client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
meta_prop_pkg_extn = {
    api_client.package_extensions.ConfigurationMetaNames.NAME: "watsonx.ai env with langgraph",
    api_client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Environment with langgraph",
    api_client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}

pkg_extn_details = api_client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="config.yaml")
pkg_extn_id = api_client.package_extensions.get_id(pkg_extn_details)
pkg_extn_id

Creating package extensions
SUCCESS


'2e990d5b-dbe5-4b0a-81e1-cede89509e82'

In [15]:
meta_prop_sw_spec = {
    api_client.software_specifications.ConfigurationMetaNames.NAME: "AI service watsonx.ai custom software specification with langgraph",
    api_client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for AI service deployment",
    api_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_id}
}

sw_spec_details = api_client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = api_client.software_specifications.get_id(sw_spec_details)
api_client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)
sw_spec_id

SUCCESS


'5ecbae0e-5c58-494d-b2bd-fe9f5eaf7de2'

Store AI service with previous created custom software specifications.

In [16]:
meta_props = {
    api_client.repository.AIServiceMetaNames.NAME: "AI service SDK with langgraph",    
    api_client.repository.AIServiceMetaNames.SOFTWARE_SPEC_ID: sw_spec_id
}
stored_ai_service_details = api_client.repository.store_ai_service(deployable_ai_service, meta_props)

In [17]:
ai_service_id = api_client.repository.get_ai_service_id(stored_ai_service_details)
ai_service_id

'cdf24a20-6c66-461a-ad56-4cad43302a64'

Create online deployment of AI service.

In [18]:
meta_props = {
    api_client.deployments.ConfigurationMetaNames.NAME: "AI service with tools",
    api_client.deployments.ConfigurationMetaNames.ONLINE: {},
}

deployment_details = api_client.deployments.create(ai_service_id, meta_props)



######################################################################################

Synchronous deployment creation for id: 'cdf24a20-6c66-461a-ad56-4cad43302a64' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
............
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='6270e0b3-ed52-4f91-9a43-a3ba8131f0ec'
-----------------------------------------------------------------------------------------------




Obtain the `deployment_id` of the previously created deployment.

In [19]:
deployment_id = api_client.deployments.get_id(deployment_details)

<a id="example"></a>
## Example of Executing an AI service.

Execute `generate` method.

In [20]:
question = "What is the total sum of the numbers 11, 13, and 20? Remember to always return the final result using the last tool message."

deployments_results = api_client.deployments.run_ai_service(
    deployment_id, {"question": question}
)

In [21]:
ai_services_pretty_print(deployments_results)

 ===== HumanMessage =====
What is the total sum of the numbers 11, 13, and 20? Remember to always return the final result using the last tool message.

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "function": {
        "arguments": "{\"a\": \"11\", \"b\": \"13\"}",
        "name": "add"
      },
      "id": "chatcmpl-tool-aa6b07d025f846679702a76bb4e323d5",
      "type": "function"
    }
  ]
}

 ===== ToolMessage =====
add
24.0

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "function": {
        "arguments": "{\"a\": \"24\", \"b\": \"20\"}",
        "name": "add"
      },
      "id": "chatcmpl-tool-286054ef43224dd28f2695e2a0eb88c7",
      "type": "function"
    }
  ]
}

 ===== ToolMessage =====
add
44.0

 ===== AIMessage =====
The total sum of the numbers 11, 13, and 20 is 44.



Execute `generate_stream` method.

In [22]:
question = "Add 2 to 5. Result multiply by 3? Result devided by 10. Always return the value from the last message to the user."

deployments_results = api_client.deployments.run_ai_service_stream(
    deployment_id, {"question": question}
)

In [23]:
ai_services_pretty_print_stream(deployments_results)

 ===== HumanMessage =====
Add 2 to 5. Result multiply by 3? Result devided by 10. Always return the value from the last message to the user.

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "id": "chatcmpl-tool-a4fd3d8941124487b5c6c93af39a943d",
      "type": "function",
      "function": {
        "name": "divide",
        "arguments": "{\"a\": \"3\", \"b\": \"10\"}"
      }
    }
  ]
}

 ===== ToolMessage =====
divide
0.3

 ===== AIMessage =====
First, we need to add 2 to 5. Then, we need to multiply the result by 3. Finally, we need to divide the result by 10. The result of the last operation is 0.3.



<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to create and deploy AI service using `ibm_watsonx_ai` SDK.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.